In [1]:
import pandas as pd
import os
import sqlalchemy as sqla
import sqlite3 as sql
from sqlite3 import Error
import re

In [7]:
####### The aim of this book is to reformat the raw data and aggregate it to a file
dire = os.getcwd()

In [39]:
####### Create a .db file (only run at first time) ########

def create_connection(db_file):
    """ create a database connection to a SQLite database """
    try:
        conn = sql.connect(db_file)
    except Error as e:
        print(e)
    finally:
        conn.close()
if __name__ == '__main__':
    create_connection(dire+"/data.db")

In [3]:
db = sql.connect(dire+"/data.db")

cursor = db.execute("SELECT * FROM sqlite_master")

In [43]:
######### textbook ##########
textbooklist = pd.read_csv(dire+'/Dataset_temp/textbook_18win.csv',header = 0)
textbooklist = textbooklist.drop(textbooklist.columns[0],axis = 1)
for c in textbooklist.columns:
    textbooklist[c] = textbooklist[c].str.replace(r'\w+:', r'')
    textbooklist[c] = textbooklist[c].str.replace(r'\s+\|\s+', r'')
textbooklist["courseid"] = textbooklist["courseid"].str.replace(r'Course', r'')
textbooklist["book_pubtime"] = textbooklist["book_pubtime"].str.replace(r'Published ', r'')
textbook_final = textbooklist.loc[textbooklist["book_ISBN"] != "book_ISBN",:]
textbook_final = textbook_final.drop_duplicates()
textbook_final = textbook_final.join(textbook_final['coursename'].str.split(expand=True).rename(columns={0:'dname', 1:'cno'}))
#textbook_final = textbook_final.reset_index()

In [47]:
textbook_final.shape

(1569, 13)

In [1]:
a =[]
a.append(":")

In [3]:
a.append("v")

In [4]:
a

[':', 'v']

In [45]:
textbook_final.head()

,term,coursename,coursesection,Instructor,courseid,book_needed,book_name,book_author,book_edition,book_pubtime,book_ISBN,dname,cno
0,Winter 18,AAS 016,001,"Mukiibi, Elizabeth",40181,Required,DEATH+THE KING'S HORSEMAN,SOYINKA,75,1975,9780393322996,AAS,016
1,Winter 18,AAS 016,001,"Mukiibi, Elizabeth",40181,Required,TRIAL OF DEDAN KIMATHI,THIONG'0,13,2013,9781478611318,AAS,016
2,Winter 18,AAS 050,ALL,"WHITE, E",,Required,BEYOND BLACKFACE,BRUNDAGE,11,2011,9780807871843,AAS,050
3,Winter 18,AAS 050,ALL,"WHITE, E",,Required,"TOMS,COONS,MULATTOES,MAMMIES,+BUCKS",BOGLE,5TH 16,2016,9780826429537,AAS,050
4,Winter 18,AAS 100,ALL,,,Required,BEHIND THE WHITE PICKET FENCE,MAYORGA-GALLO,14,2014,9781469618630,AAS,100


In [48]:
###### save textbooklist ################
textbook_final.to_sql("textbook",db,index=False)

In [63]:
######### courseCRN ##########
courseCRN = pd.read_csv(dire+'/Dataset_temp/course_CRN.csv',header = 0)
courseCRN = courseCRN.drop(courseCRN.columns[0],axis = 1)
courseCRN.head()

,CRN,Course
0,74279,AAS 015
1,74279,AAS 015
2,40181,AAS 016
3,74278,AAS 017
4,74280,AAS 050


In [49]:
###### save courseCRN ################
courseCRN.to_sql("courseCRN",db,index=False)

In [77]:
######### department ##########
deptID = pd.read_csv(dire+'/Dataset/dept.csv',header = 0)
deptID = deptID.drop(deptID.columns[0],axis = 1)
deptID["did"] = deptID["did"].str.replace("dpt-","")
deptID.head()

,did,dname
0,23459,AAS
1,25725,ABG
2,23462,ABI
3,23463,ABT
4,38037,ACC


In [78]:
###### save department ################
deptID.to_sql("deptID",db,index=False)

In [58]:
###### course detail ################
coursedtl = pd.read_csv(dire+'/Dataset_temp/course_detail.csv',header = 0)
coursedtl = coursedtl.drop(coursedtl.columns[0],axis = 1)
coursedtl = coursedtl.drop_duplicates()
coursedtl = coursedtl.reset_index(drop=True)

In [59]:
coursedtl.head()

,Available,CRN,Final,Form_GE,Instructor,L_T,Maximum_seat,New_GE,subject,term,title,unit
0,Available Seats: \r\r\n \t\t\t\t6,CRN: 74279,"Final Exam: Thursday, March 22 at 8:30 pm",Former GE Credit (Prior to Fall 2011 catalog r...,Instructor:\r\n\r\r\n\t\t\t\tJeramy Decristo,"['TR', '6:10 - 7:20 PM', 'Wellman Hall 230']",Maximum Enrollment: \r\r\n\t\t\t55,New GE Credit (Start Fall 2011 catalog rights)...,Subject Area: African American & African Std,"Term: \r\r\n\t\tWinter Quarter, 2018",AAS 015 001 - Int Afr Amer Humanities,Units: \r\r\n4
1,Available Seats: \r\r\n \t\t\t\t6,CRN: 74279,"Final Exam: Thursday, March 22 at 8:30 pm",Former GE Credit (Prior to Fall 2011 catalog r...,Instructor:\r\n\r\r\n\t\t\t\tJeramy Decristo,"['TR', '7:30 - 8:00 PM', 'Wellman Hall 230']",Maximum Enrollment: \r\r\n\t\t\t55,New GE Credit (Start Fall 2011 catalog rights)...,Subject Area: African American & African Std,"Term: \r\r\n\t\tWinter Quarter, 2018",AAS 015 001 - Int Afr Amer Humanities,Units: \r\r\n4
2,Available Seats: \r\r\n \t\t\t\t20,CRN: 40181,"Final Exam: Tuesday, March 20 at 6:00 pm",Former GE Credit (Prior to Fall 2011 catalog r...,Instructor:\r\n\r\r\n\t\t\t\tElizabeth Mukiibi,"['MW', '12:10 - 2:00 PM', 'Wellman Hall 212']",Maximum Enrollment: \r\r\n\t\t\t60,New GE Credit (Start Fall 2011 catalog rights)...,Subject Area: African American & African Std,"Term: \r\r\n\t\tWinter Quarter, 2018",AAS 016 001 - African Verbal Arts,Units: \r\r\n4
3,Available Seats: \r\r\n \t\t\t\t4,CRN: 74278,"Final Exam: Wednesday, March 21 at 8:00 am",Former GE Credit (Prior to Fall 2011 catalog r...,Instructor:\r\n\r\r\n\t\t\t\tMoradewun Adejunmobi,"['MW', '4:10 - 6:00 PM', 'Wellman Hall 202']",Maximum Enrollment: \r\r\n\t\t\t85,New GE Credit (Start Fall 2011 catalog rights)...,Subject Area: African American & African Std,"Term: \r\r\n\t\tWinter Quarter, 2018",AAS 017 001 - Women in Africa,Units: \r\r\n4
4,Available Seats: \r\r\n \t\t\t\t1,CRN: 74280,"Final Exam: Monday, March 19 at 8:00 am",Former GE Credit (Prior to Fall 2011 catalog r...,Instructor:\r\n\r\r\n\t\t\t\tElisa White,"['TR', '10:30 - 11:50 AM', 'Wellman Hall 234']",Maximum Enrollment: \r\r\n\t\t\t25,New GE Credit (Start Fall 2011 catalog rights)...,Subject Area: African American & African Std,"Term: \r\r\n\t\tWinter Quarter, 2018",AAS 050 A01 - Black Images,Units: \r\r\n4


In [60]:
coursedtl["New_GE"] = coursedtl["New_GE"].str.replace(r'(\S+\s+)+\S+:', r'')
coursedtl["Form_GE"] = coursedtl["Form_GE"].str.replace(r'(\S+\s+)+\S+:', r'')

In [61]:
###### remove the part before : #######
colk = set(coursedtl.columns)
colk.remove("L_T")
for c in colk:
    coursedtl[c] = coursedtl[c].str.replace(r'([a-zA-Z]*\s)*[a-zA-Z]*:', r'')
    coursedtl[c] = coursedtl[c].str.replace(r'\s{2,}', r'')

In [62]:
coursedtl = coursedtl.join(coursedtl['title'].str.split('-', 1, expand=True).rename(columns={0:'course', 1:'cname'}))
coursedtl = coursedtl.join(coursedtl['L_T'].str.split(',', 2, expand=True).rename(columns={0:'wkday', 1:'time',2:"location"}))
coursedtl.head()

,Available,CRN,Final,Form_GE,Instructor,L_T,Maximum_seat,New_GE,subject,term,title,unit,course,cname,wkday,time,location
0,6,74279,"Thursday, March22 at 830 pm",Arts & Humanities Diversity Writing Experience,Jeramy Decristo,"['TR', '6:10 - 7:20 PM', 'Wellman Hall 230']",55,"American Cultures, Governance & History Arts &...",African American & African Std,"Winter Quarter, 2018",AAS 015 001 - Int Afr Amer Humanities,4,AAS 015 001,Int Afr Amer Humanities,['TR','6:10 - 7:20 PM','Wellman Hall 230']
1,6,74279,"Thursday, March22 at 830 pm",Arts & Humanities Diversity Writing Experience,Jeramy Decristo,"['TR', '7:30 - 8:00 PM', 'Wellman Hall 230']",55,"American Cultures, Governance & History Arts &...",African American & African Std,"Winter Quarter, 2018",AAS 015 001 - Int Afr Amer Humanities,4,AAS 015 001,Int Afr Amer Humanities,['TR','7:30 - 8:00 PM','Wellman Hall 230']
2,20,40181,"Tuesday, March20 at 600 pm",Arts & Humanities Diversity Writing Experience,Elizabeth Mukiibi,"['MW', '12:10 - 2:00 PM', 'Wellman Hall 212']",60,Arts & Humanities Visual Literacy World Cultures,African American & African Std,"Winter Quarter, 2018",AAS 016 001 - African Verbal Arts,4,AAS 016 001,African Verbal Arts,['MW','12:10 - 2:00 PM','Wellman Hall 212']
3,4,74278,"Wednesday, March21 at 800 am",Diversity Social Sciences Writing Experience,Moradewun Adejunmobi,"['MW', '4:10 - 6:00 PM', 'Wellman Hall 202']",85,Social Sciences World Cultures Writing Experience,African American & African Std,"Winter Quarter, 2018",AAS 017 001 - Women in Africa,4,AAS 017 001,Women in Africa,['MW','4:10 - 6:00 PM','Wellman Hall 202']
4,1,74280,"Monday, March19 at 800 am",Arts & Humanities Diversity Writing Experience,Elisa White,"['TR', '10:30 - 11:50 AM', 'Wellman Hall 234']",25,"American Cultures, Governance & History Arts &...",African American & African Std,"Winter Quarter, 2018",AAS 050 A01 - Black Images,4,AAS 050 A01,Black Images,['TR','10:30 - 11:50 AM','Wellman Hall 234']


In [32]:
coursedtl = coursedtl.drop(['L_T', 'title'], axis=1)

In [42]:
coursedtl["wkday"] = coursedtl["wkday"].str.replace(r"[\[|'|\]]*", r'')
coursedtl["time"] = coursedtl["time"].str.replace(r"[\[|'|\]]*", r'')
coursedtl["location"] = coursedtl["location"].str.replace(r"[\[|'|\]]*", r'')
coursedtl["wkday"] = coursedtl["wkday"].str.replace(r"([A-Z])([A-Z])*", r'\1,\2')

In [57]:
#### split dataframe cells containing lists into multiple rows, duplicating the other column's values ####
def splitDataFrameList(df,target_column,separator):
    row_accumulator = []

    def splitListToRows(row, separator):
        split_row = row[target_column].split(separator)
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s
            row_accumulator.append(new_row)

    df.apply(splitListToRows, axis=1, args = (separator, ))
    new_df = pd.DataFrame(row_accumulator)
    return new_df

In [58]:
test = splitDataFrameList(coursedtl,"wkday",",")

In [59]:
test

,Available,CRN,Final,Form_GE,Instructor,Maximum_seat,New_GE,cname,course,location,subject,term,time,unit,wkday
0,6,74279,"Thursday, March22 at 830 pm",Arts & Humanities Diversity Writing Experience,Jeramy Decristo,55,"American Cultures, Governance & History Arts &...",Int Afr Amer Humanities,AAS 015 001,Wellman Hall 230,African American & African Std,"Winter Quarter, 2018",6:10 - 7:20 PM,4,T
1,6,74279,"Thursday, March22 at 830 pm",Arts & Humanities Diversity Writing Experience,Jeramy Decristo,55,"American Cultures, Governance & History Arts &...",Int Afr Amer Humanities,AAS 015 001,Wellman Hall 230,African American & African Std,"Winter Quarter, 2018",6:10 - 7:20 PM,4,R
2,6,74279,"Thursday, March22 at 830 pm",Arts & Humanities Diversity Writing Experience,Jeramy Decristo,55,"American Cultures, Governance & History Arts &...",Int Afr Amer Humanities,AAS 015 001,Wellman Hall 230,African American & African Std,"Winter Quarter, 2018",7:30 - 8:00 PM,4,T
3,6,74279,"Thursday, March22 at 830 pm",Arts & Humanities Diversity Writing Experience,Jeramy Decristo,55,"American Cultures, Governance & History Arts &...",Int Afr Amer Humanities,AAS 015 001,Wellman Hall 230,African American & African Std,"Winter Quarter, 2018",7:30 - 8:00 PM,4,R
4,20,40181,"Tuesday, March20 at 600 pm",Arts & Humanities Diversity Writing Experience,Elizabeth Mukiibi,60,Arts & Humanities Visual Literacy World Cultures,African Verbal Arts,AAS 016 001,Wellman Hall 212,African American & African Std,"Winter Quarter, 2018",12:10 - 2:00 PM,4,M
5,20,40181,"Tuesday, March20 at 600 pm",Arts & Humanities Diversity Writing Experience,Elizabeth Mukiibi,60,Arts & Humanities Visual Literacy World Cultures,African Verbal Arts,AAS 016 001,Wellman Hall 212,African American & African Std,"Winter Quarter, 2018",12:10 - 2:00 PM,4,W
6,4,74278,"Wednesday, March21 at 800 am",Diversity Social Sciences Writing Experience,Moradewun Adejunmobi,85,Social Sciences World Cultures Writing Experience,Women in Africa,AAS 017 001,Wellman Hall 202,African American & African Std,"Winter Quarter, 2018",4:10 - 6:00 PM,4,M
7,4,74278,"Wednesday, March21 at 800 am",Diversity Social Sciences Writing Experience,Moradewun Adejunmobi,85,Social Sciences World Cultures Writing Experience,Women in Africa,AAS 017 001,Wellman Hall 202,African American & African Std,"Winter Quarter, 2018",4:10 - 6:00 PM,4,W
8,1,74280,"Monday, March19 at 800 am",Arts & Humanities Diversity Writing Experience,Elisa White,25,"American Cultures, Governance & History Arts &...",Black Images,AAS 050 A01,Wellman Hall 234,African American & African Std,"Winter Quarter, 2018",10:30 - 11:50 AM,4,T
9,1,74280,"Monday, March19 at 800 am",Arts & Humanities Diversity Writing Experience,Elisa White,25,"American Cultures, Governance & History Arts &...",Black Images,AAS 050 A01,Wellman Hall 234,African American & African Std,"Winter Quarter, 2018",10:30 - 11:50 AM,4,R


In [62]:
test.to_sql("coursedtl",db,index=False)

In [53]:
db.execute("DROP TABLE deptabb")

In [4]:
deptabb = pd.read_excel(dire+'/Dataset_temp/deptabb.xlsx',header = 0)

In [5]:
deptabb.head()

,Subject_Code,Transcript_Subject,Subject_Description
0,AAS,AF-AMST,African American & African Std
1,ABG,ANIMBIO,Animal Biology Grad Gp
2,ABI,AN BIOL,Animal Biology
3,ABS,AP BSC,Applied Behavioral Sciences
4,ABT,ABIOSYT,Applied Biological System Tech


In [56]:
deptabb.to_sql("deptabb",db,index=False)

In [5]:
df = pd.read_csv("loca.csv")
df = df[["location","northeast","southwest","represent"]]

df = df[df.northeast != "[]"]
df = df[df.location != "  "]
df.northeast = df.northeast.str.replace(r"\[|\]","")
df.southwest = df.southwest.str.replace(r"\[|\]","")
df.represent = df.represent.str.replace(r"\[|\]","")

df = df.join(df['northeast'].str.split(',', 1, expand=True).rename(columns={0:'nlat', 1:'nlon'}))
df = df.join(df['southwest'].str.split(',', 1, expand=True).rename(columns={0:'slat', 1:'slon'}))
df = df.join(df['represent'].str.split(',', 1, expand=True).rename(columns={0:'rlat', 1:'rlon'}))

df = df.apply(pd.to_numeric, errors='ignore')

In [6]:
df.head()

,location,northeast,southwest,represent,nlat,nlon,slat,slon,rlat,rlon
0,Wellman Hall 230,"38.5426904802915, -121.7500650197085","38.5399925197085, -121.7527629802915","38.5413415, -121.751414",38.54269,-121.750065,38.539993,-121.752763,38.541341,-121.751414
1,Wellman Hall 212,"38.5433700802915, -121.7531927197085","38.5406721197085, -121.7558906802915","38.5420211, -121.7545417",38.54337,-121.753193,38.540672,-121.755891,38.542021,-121.754542
2,Wellman Hall 202,"38.5426904802915, -121.7500650197085","38.5399925197085, -121.7527629802915","38.5413415, -121.751414",38.54269,-121.750065,38.539993,-121.752763,38.541341,-121.751414
3,Wellman Hall 234,"38.5426904802915, -121.7500650197085","38.5399925197085, -121.7527629802915","38.5413415, -121.751414",38.54269,-121.750065,38.539993,-121.752763,38.541341,-121.751414
4,Wellman Hall 27,"38.5426904802915, -121.7500650197085","38.5399925197085, -121.7527629802915","38.5413415, -121.751414",38.54269,-121.750065,38.539993,-121.752763,38.541341,-121.751414


In [7]:
df.to_sql("location",db,index=False)

In [10]:
deptstem = pd.read_excel(dire+'/Dataset_temp/deptdiv.xlsx',header = 0)

deptstem.head()

,department,detail,Category
0,PHY,Physics,STEM
1,IST,Integrated Studies,ART
2,NSC,Neuroscience (Graduate Group),STEM
3,PSC,Psychology,STEM
4,PBI,Plant Biology (Graduate Group),STEM


In [11]:
deptstem.to_sql("deptstem",db,index=False)

In [12]:
pd.read_sql("SELECT * FROM deptstem LIMIT 10",db)

,department,detail,Category
0,PHY,Physics,STEM
1,IST,Integrated Studies,ART
2,NSC,Neuroscience (Graduate Group),STEM
3,PSC,Psychology,STEM
4,PBI,Plant Biology (Graduate Group),STEM
5,MAT,Mathematics,STEM
6,ECS,Engineering Computer Science,STEM
7,CHI,Chicano Studies,ART
8,NEM,Nematology,STEM
9,UWP,University Writing Program,ART


In [17]:
ratemyprof = pd.read_csv(dire+'/Dataset_temp/ratemyprofessor.csv',lineterminator='\n')
ratemyprof = ratemyprof.drop(ratemyprof.columns[0],axis = 1)
ratemyprof.head()

,Attendance,Class,Department,For Credit,Grade Received,Level of Difficulty,Overall Quality,Professor,RatingType,Textbook Used,Would Take Again,comment,comment tags,date
0,N/A,HIS188,the History department,Yes,N/A,3.0,5.0,Rauchway Eric,awesome,Yes,Yes,"No doubt, Rauchway is a solid lecturer and the...",[''],12/20/2017\r\r
1,Not Mandatory,HIS119,the History department,Yes,A,3.0,4.0,Rauchway Eric,good,Yes,Yes,Teaching a joint course with Prof. Campbell on...,"['BEWARE OF POP QUIZZES', 'RESPECTED', 'AMAZIN...",06/21/2017\r\r
2,Mandatory,HIS174B,the History department,Yes,N/A,3.0,2.0,Rauchway Eric,poor,Yes,No,He is unorganized and offers random in-class q...,"['PARTICIPATION MATTERS', 'TOUGH GRADER', ""SKI...",09/30/2016\r\r
3,Not Mandatory,HIS017B,the History department,Yes,A,3.0,4.0,Rauchway Eric,good,Yes,Yes,Pretty solid and entertaining lecturer. Howeve...,"['EXTRA CREDIT', 'GRADED BY FEW THINGS', 'GET ...",06/16/2016\r\r
4,Mandatory,HIS188,the History department,Yes,B+,3.0,4.5,Rauchway Eric,awesome,No,N/A,I really enjoyed the lectures. The books were ...,"['AMAZING LECTURES', 'CLEAR GRADING CRITERIA',...",12/31/2015\r\r


In [19]:
ratemyprof.to_sql("rmp",db,index=False)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:1534: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


In [22]:
pd.read_sql("SELECT * FROM rmp LIMIT 10",db)

,Attendance,Class,Department,For Credit,Grade Received,Level of Difficulty,Overall Quality,Professor,RatingType,Textbook Used,Would Take Again,comment,comment tags,date
0,N/A,HIS188,the History department,Yes,N/A,3.0,5.0,Rauchway Eric,awesome,Yes,Yes,"No doubt, Rauchway is a solid lecturer and the...",[''],12/20/2017\r\r
1,Not Mandatory,HIS119,the History department,Yes,A,3.0,4.0,Rauchway Eric,good,Yes,Yes,Teaching a joint course with Prof. Campbell on...,"['BEWARE OF POP QUIZZES', 'RESPECTED', 'AMAZIN...",06/21/2017\r\r
2,Mandatory,HIS174B,the History department,Yes,N/A,3.0,2.0,Rauchway Eric,poor,Yes,No,He is unorganized and offers random in-class q...,"['PARTICIPATION MATTERS', 'TOUGH GRADER', ""SKI...",09/30/2016\r\r
3,Not Mandatory,HIS017B,the History department,Yes,A,3.0,4.0,Rauchway Eric,good,Yes,Yes,Pretty solid and entertaining lecturer. Howeve...,"['EXTRA CREDIT', 'GRADED BY FEW THINGS', 'GET ...",06/16/2016\r\r
4,Mandatory,HIS188,the History department,Yes,B+,3.0,4.5,Rauchway Eric,awesome,No,N/A,I really enjoyed the lectures. The books were ...,"['AMAZING LECTURES', 'CLEAR GRADING CRITERIA',...",12/31/2015\r\r
5,Not Mandatory,HIS120,the History department,Yes,A,2.0,4.5,Rauchway Eric,awesome,Yes,Yes,Rauchway's WWII class inspired me to take more...,"['INSPIRATIONAL', 'AMAZING LECTURES']",11/05/2015\r\r
6,Not Mandatory,HIS120,the History department,Yes,A,3.0,3.0,Rauchway Eric,average,Yes,N/A,Professor Rauchway's WWII class has been by f...,"['AMAZING LECTURES', 'GET READY TO READ', ""SKI...",09/09/2015\r\r
7,N/A,HIS120,the History department,Yes,N/A,3.0,3.5,Rauchway Eric,good,Yes,Yes,He was honestly the best professor I have had ...,"['GET READY TO READ', 'AMAZING LECTURES']",04/01/2015\r\r
8,Not Mandatory,HIS120,the History department,Yes,A-,1.0,4.0,Rauchway Eric,good,Yes,Yes,"Lots of memorization but, overall, it was a re...","['CLEAR GRADING CRITERIA', 'GET READY TO READ']",12/24/2014\r\r
9,Not Mandatory,HIS120,the History department,Yes,B,3.0,4.0,Rauchway Eric,good,No,Yes,This guy knows how to make history interesting...,['AMAZING LECTURES'],12/12/2014\r\r


In [9]:
course = pd.read_csv(dire+'/Dataset_temp/course.csv')
course = course.drop(course.columns[0],axis = 1)
course.head()

,did,cid,cname
0,23459,9310202,"015 -001 -Decristo, Jeramy"
1,23459,9310203,"016 -001 -Mukiibi, Elizabeth"
2,23459,9310204,"017 -001 -Adejunmobi, Moradewu"
3,23459,9304058,"050 -ALL -WHITE, E"
4,23459,9310209,099 -000 -The Staff


In [11]:
course.to_sql("course",db,index=False)

In [12]:
pd.read_sql("SELECT * FROM sqlite_master",db)

,type,name,tbl_name,rootpage,sql
0,table,courseCRN,courseCRN,4,"CREATE TABLE ""courseCRN"" (\n""CRN"" TEXT,\n ""Co..."
1,table,deptID,deptID,186,"CREATE TABLE ""deptID"" (\n""did"" TEXT,\n ""dname..."
2,table,coursedtl,coursedtl,187,"CREATE TABLE ""coursedtl"" (\n""Available"" TEXT,\..."
3,table,location,location,1023,"CREATE TABLE ""location"" (\n""location"" TEXT,\n ..."
4,table,textbook,textbook,2,"CREATE TABLE ""textbook"" (\n""term"" TEXT,\n ""co..."
5,table,deptabb,deptabb,9,"CREATE TABLE ""deptabb"" (\n""Subject_Code"" TEXT,..."
6,table,deptstem,deptstem,11,"CREATE TABLE ""deptstem"" (\n""department"" TEXT,\..."
7,table,rmp,rmp,13,"CREATE TABLE ""rmp"" (\n""Attendance"" TEXT,\n ""C..."
8,table,course,course,3111,"CREATE TABLE ""course"" (\n""did"" INTEGER,\n ""ci..."
